In [7]:
#import random
import numpy as np
import pandas as pd

In [2]:


#                             0    1+   31+   61+   91+   WOF
dod_migration = np.array([[0.95, 0.05, 0.00, 0.00, 0.00, 0.00], #  0 
                          [0.90, 0.05, 0.05, 0.00, 0.00, 0.00], #  1+
                          [0.10, 0.05, 0.05, 0.80, 0.00, 0.00], # 31+
                          [0.05, 0.05, 0.05, 0.05, 0.80, 0.00], # 61+
                          [0.01, 0.01, 0.02, 0.02, 0.04, 0.90], # 91+
                          [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # WOF
                         ])
[i.sum() for i in dod_migration]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [73]:
class Contract():
    """Class Contract
       issue_dt - issue of contract
       duration - duration in months
    """

    dod_dic = {0: '0',
               1: '1+',
               2: '31+',
               3: '61+',
               4: '91+',
               5: 'WOF'
              }
    dod_cnt = 6 # кол-во состояний
    dod_states = np.eye(dod_cnt) # матрица состояний (для удобства использована единичная матрица)

    def __init__(self, cntr_id = 0, issue_dt = 0, duration = 0,
                 dod_migration = None):
        self.cntr_id = cntr_id
        self.dod_id = 0        # начальное состояние контракта при выдачи: DOD = 0
        self.dod_state = self.dod_states[0] # np.array([1,0,0,0,0]) 
        self.dod_migration = dod_migration
        self.issue_dt = issue_dt
        self.mob = 0
        self.duration = duration
        self.closed_id = 0     # 0 - контратк открыт, 1 - закрыт
        
    def next_month(self):
        if self.closed_id == 1:
            return None
           
        self.mob = self.mob + 1
        p = self.dod_migration.T.dot(self.dod_state) # array of probabilities
        self.dod_id = np.random.choice(self.dod_cnt,1,p=p)[0] # new state
        self.dod_state = self.dod_states[self.dod_id]

        if self.dod_id == 0 and self.mob > self.duration: # погашение либо выздоровление с возвращением в график
            self.closed_id = 1
        
        if self.dod_id == 5 and self.mob > 50: # списание
            self.closed_id = 1
        

In [5]:
class World():
    """Class World - Макромир, который задает начало отсчета времени, законы макроэкономики и остальное окружение."""

    def __init__(self):
        self.World_Time = 0

In [8]:
class DWH_DB():
    """Class DWH - база данных
    """
    def __init__(sefl):
        sefl.LI = pd.DataFrame(columns = ['CNTR_ID',
                                          'SD',
                                          'DOD_ID',
                                          'MOB'
                                          ])

DWH = DWH_DB()

In [53]:
class Portfolio():
    """Class Portfolio - Портфель - динамика 
    N - первая выдача при создании портфеля
    start_portfolio_dt - привязка портфеля к мировому времени - важно при наличии нескольких портфелей
    
    """
    def __init__(self, N = 10, start_portfolio_dt = 0):
        self.cntr_id = 0                                # счетчик контрактов
        self.start_portfolio_dt = start_portfolio_dt    # дата создания портфеля
        self.cntr_list = []                             # сам портфель
        self.portfolio_age = 0                          # возрвст портфеля

        # проведем выдачу
        self.issue(N)

    def issue(self, N = 10):
        for i in range(N):
            self.cntr_id += 1
            self.cntr_list.append(Contract(cntr_id = self.cntr_id, issue_dt = self.start_portfolio_dt, duration = 36, dod_migration = dod_migration))

    def next_month(self, N = 10):
        self.portfolio_age +=1
        # сдвинем существующий портфель, потом проведем выдачу новых 
        for cntr in self.cntr_list:
            cntr.next_month()

        self.issue(N)

    def fix_in_dwh(self):
        DWH.LI.append()

# journal
# tinkoff
# ru
# how-to-choose-yacht-school

In [78]:
GP = Portfolio()

In [79]:
#GP.next_month(10)

In [80]:
for t in range(300):
    cnt = 0
    for c in GP.cntr_list:
        if c.closed_id == 0:
            cnt +=1
    print('%04i'%GP.portfolio_age, cnt)
    GP.next_month(10)

0000 10
0001 20
0002 30
0003 40
0004 50
0005 60
0006 70
0007 80
0008 90
0009 100
0010 110
0011 120
0012 130
0013 140
0014 150
0015 160
0016 170
0017 180
0018 190
0019 200
0020 210
0021 220
0022 230
0023 240
0024 250
0025 260
0026 270
0027 280
0028 290
0029 300
0030 310
0031 320
0032 330
0033 340
0034 350
0035 360
0036 370
0037 373
0038 374
0039 375
0040 376
0041 375
0042 375
0043 377
0044 378
0045 378
0046 380
0047 381
0048 382
0049 382
0050 384
0051 383
0052 382
0053 383
0054 382
0055 384
0056 384
0057 382
0058 380
0059 381
0060 380
0061 381
0062 381
0063 381
0064 380
0065 380
0066 380
0067 380
0068 379
0069 379
0070 379
0071 379
0072 379
0073 380
0074 380
0075 377
0076 376
0077 376
0078 375
0079 375
0080 377
0081 378
0082 375
0083 378
0084 377
0085 377
0086 376
0087 376
0088 375
0089 376
0090 378
0091 378
0092 379
0093 379
0094 380
0095 381
0096 380
0097 378
0098 379
0099 379
0100 380
0101 380
0102 381
0103 381
0104 379
0105 380
0106 378
0107 380
0108 381
0109 382
0110 381
0111 381
0

In [41]:
cntr = Contract(cntr_id = 1, issue_dt = 0, duration = 36, dod_migration = dod_migration)

In [42]:
for i in range(cntr.duration + 10):
    cntr.next_month()
    print(cntr.mob, cntr.dod_state, cntr.closed_id)

1 [1. 0. 0. 0. 0.] 0
2 [1. 0. 0. 0. 0.] 0
3 [1. 0. 0. 0. 0.] 0
4 [1. 0. 0. 0. 0.] 0
5 [1. 0. 0. 0. 0.] 0
6 [1. 0. 0. 0. 0.] 0
7 [1. 0. 0. 0. 0.] 0
8 [1. 0. 0. 0. 0.] 0
9 [1. 0. 0. 0. 0.] 0
10 [1. 0. 0. 0. 0.] 0
11 [1. 0. 0. 0. 0.] 0
12 [1. 0. 0. 0. 0.] 0
13 [1. 0. 0. 0. 0.] 0
14 [1. 0. 0. 0. 0.] 0
15 [1. 0. 0. 0. 0.] 0
16 [1. 0. 0. 0. 0.] 0
17 [0. 1. 0. 0. 0.] 0
18 [1. 0. 0. 0. 0.] 0
19 [1. 0. 0. 0. 0.] 0
20 [1. 0. 0. 0. 0.] 0
21 [1. 0. 0. 0. 0.] 0
22 [1. 0. 0. 0. 0.] 0
23 [1. 0. 0. 0. 0.] 0
24 [1. 0. 0. 0. 0.] 0
25 [1. 0. 0. 0. 0.] 0
26 [1. 0. 0. 0. 0.] 0
27 [0. 1. 0. 0. 0.] 0
28 [0. 0. 1. 0. 0.] 0
29 [0. 0. 0. 1. 0.] 0
30 [0. 1. 0. 0. 0.] 0
31 [0. 0. 1. 0. 0.] 0
32 [0. 0. 0. 1. 0.] 0
33 [0. 0. 0. 0. 1.] 0
34 [0. 0. 0. 0. 1.] 0
35 [0. 0. 0. 0. 1.] 0
36 [0. 0. 0. 0. 1.] 0
37 [0. 0. 0. 0. 1.] 0
38 [0. 0. 0. 0. 1.] 0
39 [0. 0. 0. 0. 1.] 0
40 [0. 0. 0. 0. 1.] 0
41 [0. 0. 0. 0. 1.] 0
42 [0. 0. 0. 0. 1.] 0
43 [0. 0. 0. 0. 1.] 0
44 [0. 0. 0. 0. 1.] 0
45 [0. 0. 0. 0. 1.] 0
46 [0. 0. 0. 0. 1.]